In [3]:
#!pip install wordcloud
import json
import pandas as pd 
import matplotlib.pyplot as plt 
from pprint import pprint # pretty print
from tqdm import tqdm # google this
from wordcloud import WordCloud
import itertools
from collections import Counter
import collections
import csv
import numpy as np

In [18]:
#seattle = []
#with open("listings.csv","r") as fp:
    #for line in tqdm(fp):
        #try:
            #seattle.append(csv.loads(line))
        #except:
            #pass
#df = pd.DataFrame.from_dict(seattle)
#pprint(seattle)


In [21]:
listings_df = pd.read_csv('listings.csv')
listings_df

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,2318,https://www.airbnb.com/rooms/2318,20210318170758,2021-04-01,Casa Madrona - Urban Oasis 1 block from the park!,Now booking 30 night Minimum only. Please cont...,Madrona is a hidden gem of a neighborhood. It ...,https://a0.muscache.com/pictures/02973ad3-a7a3...,2536,https://www.airbnb.com/users/show/2536,...,10.0,10.0,10.0,STR-OPLI-19-002837,f,2,2,0,0,0.21
1,9419,https://www.airbnb.com/rooms/9419,20210318170758,2021-03-19,Glorious sun room w/ memory foambed,"Keeping you safe is our priority, we are adher...","Lots of restaurants (see our guide book) bars,...",https://a0.muscache.com/pictures/56645186/e5fb...,30559,https://www.airbnb.com/users/show/30559,...,10.0,10.0,10.0,STR-OPLI-19-003039,f,7,0,7,0,1.14
2,9531,https://www.airbnb.com/rooms/9531,20210318170758,2021-03-20,The Adorable Sweet Orange Craftsman,The Sweet Orange is a delightful and spacious ...,The neighborhood is awesome! Just far enough ...,https://a0.muscache.com/pictures/30470355/052c...,31481,https://www.airbnb.com/users/show/31481,...,10.0,10.0,10.0,STR-OPLI-19-002182,f,2,2,0,0,0.37
3,9534,https://www.airbnb.com/rooms/9534,20210318170758,2021-03-19,The Coolest Tangerine Dream MIL!,Welcome to my delicious Tangerine Dream! A co...,The neighborhood is the best of two worlds...w...,https://a0.muscache.com/pictures/30476721/0751...,31481,https://www.airbnb.com/users/show/31481,...,10.0,10.0,10.0,STR-OPLI-19-002182,f,2,2,0,0,0.50
4,9596,https://www.airbnb.com/rooms/9596,20210318170758,2021-03-20,"the down home , spacious, central and fab!","We are in a great neighborhood, quiet, full of...","if you arrive early for check in at 3, I reco...",https://a0.muscache.com/pictures/665252/102d18...,14942,https://www.airbnb.com/users/show/14942,...,9.0,10.0,9.0,STR-OPLI-19-002622,f,2,2,0,0,0.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4208,48715255,https://www.airbnb.com/rooms/48715255,20210318170758,2021-03-19,Spacious 2BD apartment with self-checkin,Experience a home away from home in these mode...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,8534462,https://www.airbnb.com/users/show/8534462,...,NaN,NaN,NaN,STR-OPLI-19-003325,t,140,140,0,0,NaN
4209,48715257,https://www.airbnb.com/rooms/48715257,20210318170758,2021-03-20,"Amazing 2BD Seattle apartment, fast wifi, self...",Experience a home away from home in these mode...,NaN,https://a0.muscache.com/pictures/prohost-api/H...,8534462,https://www.airbnb.com/users/show/8534462,...,NaN,NaN,NaN,STR-OPLI-19-003325,t,140,140,0,0,NaN
4210,48718112,https://www.airbnb.com/rooms/48718112,20210318170758,2021-03-19,Seattle Water View Home with Hot Tub: Trendy B...,We want you to know we are doing our part to e...,Ballard,https://a0.muscache.com/pictures/60391490-0a5c...,8009691,https://www.airbnb.com/users/show/8009691,...,NaN,NaN,NaN,STR-OPLI-19-001991,f,5,5,0,0,NaN
4211,48718428,https://www.airbnb.com/rooms/48718428,20210318170758,2021-03-19,**WOW** Stylish Ballard True 1BR Apt - Pets ok!,Focused exclusively on 30 day+ stays to meet t...,Located right near Market Street and 15th at t...,https://a0.muscache.com/pictures/d9c8ad96-c7af...,218805459,https://www.airbnb.com/users/show/218805459,...,NaN,NaN,NaN,NaN,t,60,60,0,0,NaN


In [42]:
names = listings_df['name']
description = listings_df['description']
nbhood_oview = listings_df['neighborhood_overview']
instant_book = listings_df['instant_bookable']
lstgs_per_host = listings_df['host_total_listings_count']
host_name = listings_df['host_name']
resp_time = listings_df['host_response_time']
accep_rate = listings_df['host_acceptance_rate']
resp_rate = listings_df['host_response_rate']
superhost = listings_df['host_is_superhost']
pro_pic = listings_df['host_has_profile_pic']
nbhood = listings_df['neighbourhood_group_cleansed']
prop_type = listings_df['property_type']
num_accom = listings_df['accommodates']
baths = listings_df['bathrooms_text']
bedrooms = listings_df['bedrooms']
beds = listings_df['beds']
amenities = listings_df['amenities']
price = listings_df['price']
min_nights = listings_df['minimum_nights']
num_reviews = listings_df['number_of_reviews']
rev_scores = listings_df['review_scores_rating']
rev_per_month = listings_df['reviews_per_month']

In [32]:
names

0       Casa Madrona - Urban Oasis 1 block from the park!
1                     Glorious sun room w/ memory foambed
2                     The Adorable Sweet Orange Craftsman
3                        The Coolest Tangerine Dream MIL!
4              the down home , spacious, central and fab!
                              ...                        
4208             Spacious 2BD apartment with self-checkin
4209    Amazing 2BD Seattle apartment, fast wifi, self...
4210    Seattle Water View Home with Hot Tub: Trendy B...
4211      **WOW** Stylish Ballard True 1BR Apt - Pets ok!
4212            Capitol Hill Studio, w/ W/D by Blueground
Name: name, Length: 4213, dtype: object

In [43]:
host_name.value_counts()

Barsala           140
Kia                96
Melissa            94
Blueground         93
Eric               79
                 ... 
Blake & Lauren      1
Cyndy               1
Ashly               1
Jim                 1
Siri                1
Name: host_name, Length: 1330, dtype: int64

listings per host don't match occurances of host name because hosts might have properties in other cities

In [44]:
description[0]

"Now booking 30 night Minimum only. Please contact for seasonal discounts. <br /><br />Gorgeous, architect remodeled, Dutch Colonial in the heart of Madrona. 4 Bedroom, 2.5 bath, Chef's Kitchen, gracious living and dining room, enclosed back garden, deck, grill, fruit trees. <br /><br />2 blocks to bus. <br />3 mins to Cap Hill. <br />5 Mins to downtown<br />5 mins to Amazon, Google, South Lake Union. <br />Within 5-10 minutes of 5 of the areas largest hospitals.<br />Extra storage space in basement for families that are remodeling/moving<br /><br /><b>The space</b><br />This beautiful, gracious home has been completely restored from the studs up, adding period touches such as window benches and floor to ceiling bookshelves. The house is perfect for family gatherings, large or small, groups of friends, or colleagues.<br /><br />Upstairs are 4 bedrooms and two full baths with Olympic mountain and city skyline views. The main floor features a formal entry leading to a sun-lit living room

In [34]:
nbhood_oview

0       Madrona is a hidden gem of a neighborhood. It ...
1       Lots of restaurants (see our guide book) bars,...
2       The neighborhood is awesome!  Just far enough ...
3       The neighborhood is the best of two worlds...w...
4       if you arrive early for check in at 3,  I reco...
                              ...                        
4208                                                  NaN
4209                                                  NaN
4210                                              Ballard
4211    Located right near Market Street and 15th at t...
4212    This furnished apartment is situated in Capito...
Name: neighborhood_overview, Length: 4213, dtype: object

In [35]:
instant_book

0       f
1       f
2       f
3       f
4       f
       ..
4208    t
4209    t
4210    f
4211    t
4212    t
Name: instant_bookable, Length: 4213, dtype: object

In [40]:
lstgs_per_host

0          2.0
1          8.0
2          0.0
3          0.0
4          5.0
         ...  
4208     882.0
4209     882.0
4210      10.0
4211      10.0
4212    1435.0
Name: host_total_listings_count, Length: 4213, dtype: float64

In [7]:
neighbourhoods_df = pd.read_csv('neighbourhoods.csv')
neighbourhoods_df

,neighbourhood_group,neighbourhood
0,Ballard,Adams
1,Ballard,Loyal Heights
2,Ballard,Sunset Hill
3,Ballard,West Woodland
4,Ballard,Whittier Heights
...,...,...
85,West Seattle,Fauntleroy
86,West Seattle,Gatewood
87,West Seattle,Genesee
88,West Seattle,North Admiral


In [8]:
reviews_df = pd.read_csv('reviews.csv')
reviews_df

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2318,146,2008-09-15,2451,Kevin,1000 times better than staying at a hotel.
1,2318,126302712,2017-01-10,12332845,Jessica,"Our family (two couples, a two year old and an..."
2,2318,140977084,2017-04-01,4789466,Ivan,Top of the list locations we have stayed at! T...
3,2318,147262504,2017-04-25,55817131,Mike,"SUCH an awesome place. Very clean, quiet and s..."
4,2318,161806368,2017-06-18,113604590,Pete,We flew quite a distance to be at our only dau...
...,...,...,...,...,...,...
262464,48382242,736787217,2021-03-06,215409003,Kelvin,“They weren’t kidding when it says Pike Place ...
262465,48382242,737302569,2021-03-07,62957433,Evan,This is such a steal of a deal it’s not even f...
262466,48382242,738367090,2021-03-12,164562404,Sarah,One of the best parts about staying here was t...
262467,48382242,739244211,2021-03-14,187191376,Arianne,Amazing place for a Seattle trip. We fell in l...


In [39]:
def name_to_name_text(names):
    try:
        return names["text"]
    except KeyError: # if there is no `text` key in the dictionary
        return ""

names_list = [name_to_name_text(names).split() 
                    for name in tqdm(names)] # a list of lists

all_name_words = " ".join(list(itertools.chain.from_iterable(names_list)))

wordcloud = WordCloud(width = 500, height = 500, 
    background_color ='white', 
    #stopwords = stopwords, 
    min_font_size = 10).generate(all_name_words) 

# plot the WordCloud image					 
plt.figure(figsize = (10, 10), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 



100%|██████████| 4213/4213 [00:00<00:00, 82281.85it/s]


ValueError: We need at least 1 word to plot a word cloud, got 0.